In [1]:
import numpy as np
df=np.load("/Users/rohitviswam/Downloads/CSI-02-0005/H_32T4R_30_1RB.npy")
data=df.reshape(2100,398,256)
data.shape
feature_len=data.shape[-1]

In [2]:
data.shape

(2100, 398, 256)

In [3]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [4]:
data_flattened = data.reshape(data.shape[0], -1)  # shape becomes (2100, 398*256)


In [5]:
data = data_flattened

In [6]:
import numpy as np

# Assuming `data` is already loaded as a numpy array with shape (2100, 101888)
num_UEs = data.shape[0]
num_time_steps = data.shape[1]
input_length = 10

# Prepare lists to hold the input-output pairs for each UE
all_inputs = []
all_targets = []

# Iterate through each UE
for ue_idx in range(num_UEs):
    ue_data = data[ue_idx]

    ue_inputs = []
    ue_targets = []

    # Generate input-target pairs
    for start_idx in range(num_time_steps - input_length):
        end_idx = start_idx + input_length
        input_seq = ue_data[start_idx:end_idx]
        target_value = ue_data[end_idx]

        ue_inputs.append(input_seq)
        ue_targets.append(target_value)

    # Convert lists to numpy arrays and add to the main lists
    all_inputs.append(np.array(ue_inputs))
    all_targets.append(np.array(ue_targets))

# Now `all_inputs` and `all_targets` will each be lists containing numpy arrays of inputs and targets for each UE


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

# Assuming `data` is already loaded as a numpy array with shape (2100, 101888)
num_UEs = data.shape[0]
num_time_steps = data.shape[1]

# Set the number of samples to analyze
num_samples_to_plot = 5

# Choose random UEs to visualize
random_indices = random.sample(range(num_UEs), num_samples_to_plot)

# Visualize the time series data of a few UEs
fig, axs = plt.subplots(num_samples_to_plot, 1, figsize=(12, 18), sharex=True)
for i, idx in enumerate(random_indices):
    axs[i].plot(data[idx, :], label=f'UE {idx}')
    axs[i].set_title(f'Time Series Data for UE {idx}')
    axs[i].set_xlabel('Time Step')
    axs[i].set_ylabel('Value')
    axs[i].legend()
plt.tight_layout()
plt.show()

# Plot the distribution of values across all UEs
flat_data = data.flatten()
plt.figure(figsize=(10, 6))
sns.histplot(flat_data, bins=50, kde=True)
plt.title('Distribution of All Values')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()

# Plot the mean and variance for each UE
mean_values = np.mean(data, axis=1)
var_values = np.var(data, axis=1)

plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
plt.plot(mean_values, 'o-')
plt.title('Mean Value per UE')
plt.xlabel('UE Index')
plt.ylabel('Mean Value')

plt.subplot(2, 1, 2)
plt.plot(var_values, 'o-')
plt.title('Variance per UE')
plt.xlabel('UE Index')
plt.ylabel('Variance')
plt.tight_layout()
plt.show()

# Analyze autocorrelation of a random UE
from pandas.plotting import autocorrelation_plot

random_ue = random.choice(range(num_UEs))
plt.figure(figsize=(10, 6))
autocorrelation_plot(data[random_ue, :])
plt.title(f'Autocorrelation for UE {random_ue}')
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.show()


In [18]:
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

# Assuming data has shape (2100, 101888), where the first 10 values are inputs and the 11th value is the target
num_UEs = data.shape[0]
input_length = 10

# Simplified ARIMA order for faster training
arima_order = (1, 0, 1)  # Example: p=1, d=0, q=1

# Lists to store evaluation results
mse_results = []

# Progress bar for all UEs
for ue_idx in tqdm(range(num_UEs), desc="Processing UEs", unit="UE"):
    ue_data = data[ue_idx]

    # Split into training (80%) and testing (20%) sets
    split_idx = int(0.8 * len(ue_data))
    train_data = ue_data[:split_idx]
    test_data = ue_data[split_idx:]

    # Initialize variables to store predictions
    history = list(train_data)
    predictions = []

    # Process predictions in each UE
    for t in range(len(test_data)):
        # Define ARIMA model with the simplified order
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit()

        # Forecast the next value
        forecast = model_fit.forecast()[0]
        predictions.append(forecast)

        # Add actual value to history
        history.append(test_data[t])

        # Optionally, print progress every 10% to see where we are
        if t % (len(test_data) // 10 + 1) == 0:
            tqdm.write(f'UE {ue_idx}: Progress {t / len(test_data) * 100:.1f}%')

    # Calculate mean squared error for this UE
    mse = mean_squared_error(test_data, predictions)
    mse_results.append(mse)
    tqdm.write(f'UE {ue_idx}: Mean Squared Error = {mse:.3f}')

# Output average MSE across all UEs
average_mse = np.mean(mse_results)
print(f'Average Mean Squared Error across all UEs: {average_mse:.3f}')

Processing UEs:   0%|          | 0/2100 [00:06<?, ?UE/s]

UE 0: Progress 0.0%


In [ ]:
import pandas as pd

# Function to tabulate actual and predicted values for one UE
def tabulate_predictions(ue_idx):
    ue_data = data[ue_idx]
    split_idx = int(0.8 * len(ue_data))
    train_data = ue_data[:split_idx]
    test_data = ue_data[split_idx:]

    # Initialize history from training data
    history = list(train_data)
    predictions = []

    # Train and forecast for each test point
    for t in range(len(test_data)):
        model = ARIMA(history, order=(1, 0, 1))
        model_fit = model.fit()

        # Forecast the next value
        forecast = model_fit.forecast()[0]
        predictions.append(forecast)

        # Add actual value to history
        history.append(test_data[t])

    # Create a DataFrame to compare actual and predicted values
    results_df = pd.DataFrame({
        'Actual': test_data,
        'Predicted': predictions
    })

    return results_df

# Example usage for a single UE (replace `0` with a specific UE index to inspect)
ue_idx_to_inspect = 0
predictions_table = tabulate_predictions(ue_idx_to_inspect)

# Display the first few rows of the predictions table
print(predictions_table.head())


In [ ]:
import matplotlib.pyplot as plt

# Function to visualize the actual and predicted values for one UE
def visualize_predictions(ue_idx):
    ue_data = data[ue_idx]
    split_idx = int(0.8 * len(ue_data))
    train_data = ue_data[:split_idx]
    test_data = ue_data[split_idx:]

    # Initialize history from training data
    history = list(train_data)
    predictions = []

    # Train and forecast for each test point
    for t in range(len(test_data)):
        model = ARIMA(history, order=(1, 0, 1))
        model_fit = model.fit()

        # Forecast the next value
        forecast = model_fit.forecast()[0]
        predictions.append(forecast)

        # Add actual value to history
        history.append(test_data[t])

    # Plot the actual and predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(test_data, label='Actual', marker='o', linestyle='--')
    plt.plot(predictions, label='Predicted', marker='x', linestyle='-')
    plt.title(f'UE {ue_idx}: Actual vs. Predicted')
    plt.xlabel('Time Step')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# Example usage for a single UE (replace `0` with a specific UE index to visualize)
visualize_predictions(0)